In [1]:
from __future__ import print_function    # (at top of module)
import sys
import time
import requests

In [16]:
def setCounterParty(bank_id, account_id,counterparty_id, counterparty_iban):
    global COUNTERPARTY_BANK, OUR_COUNTERPARTY, OUR_COUNTERPARTY_ID, OUR_COUNTERPARTY_IBAN
    COUNTERPARTY_BANK = bank_id
    OUR_COUNTERPARTY = account_id
    OUR_COUNTERPARTY_ID = counterparty_id
    OUR_COUNTERPARTY_IBAN = counterparty_iban
    
def setPaymentDetails(currency,value):
    global OUR_CURRENCY, OUR_VALUE
    
    OUR_CURRENCY = currency
    OUR_VALUE =value
    
def setBaseUrl(u):
    global BASE_URL
    BASE_URL = u

def setToken(t):
    global DL_TOKEN 
    DL_TOKEN = { 'Authorization' : 'DirectLogin token=%s' % t}

def setApiVersion(v):
    global API_VERSION
    API_VERSION = v

# Helper function to merge headers
def mergeHeaders(x, y):
    z = x.copy()
    z.update(y)
    return z

# Logger
def log(m):
    if LOGGING:
        print(m)

# Login as user
def login(username, password, consumer_key):
    login_url = '{0}/my/logins/direct'.format(BASE_URL)
    login_header  = { 'Authorization' : 'DirectLogin username="%s",password="%s",consumer_key="%s"' % (username, password, consumer_key)}
    # Login and receive authorized token
    log('Login as {0} to {1}'.format(login_header, login_url))
    r = requests.post(login_url, headers=login_header)
    if (r.status_code != 201):
        log("error: could not login")
        log("text: " + r.text)
        return r.text
    t = r.json()['token']
    log("Received token: {0}".format(t))
    setToken(t)
    return t

# Request a meeting
def requestMeeting(purpose_id, provider_id):
    post_data = {
        'purpose_id'   : '%s' % purpose_id,
        'provider_id'  : '%s' % provider_id
    }
    # Send post request with attached json
    response = requests.post(u"{0}/obp/{1}/banks/THE_BANK/meetings".format(BASE_URL, API_VERSION), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    # Print result
    log("code=" + response.status_code + " text=" + response.text)
    return response.json()

def getCounterBankId():
    return COUNTERPARTY_BANK

def getCounterpartyAccountId():
    return OUR_COUNTERPARTY

def getCounterpartyId():
    return OUR_COUNTERPARTY_ID

def getCounterpartyIban():
    return OUR_COUNTERPARTY_IBAN

# Get banks
def getBanks():
    # Prepare headers
    response = requests.get(u"{0}/obp/{1}/banks".format(BASE_URL, API_VERSION), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()['banks']

# Get currently logged in user
def getCurrentUser():
    # Prepare headers
    response = requests.get(u"{0}/obp/{1}/users/current".format(BASE_URL, API_VERSION), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

# Create an user
def createUser(fname, lname, email, username, password):
    post_data = {
        'last_name'  : '%s' % lname,
        'first_name' : '%s' % fname,
        'email'      : '%s' % email,
        'username'   : '%s' % username,
        'password'   : '%s' % password 
    }
    # Send post request with attached json
    response = requests.post(u"{0}/obp/{1}/users".format(BASE_URL, API_VERSION), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    # Log result
    log("code=" + response.status_code + " text=" + response.text)
    return response.json()

# Get all user's private accounts
def getPrivateAccounts(bank):
    # Prepare headers
    response = requests.get(u"{0}/obp/{1}/banks/{2}/accounts/private".format(BASE_URL, API_VERSION, bank), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()['accounts']

# Get a single account
def getAccount(bank, account):
    # Prepare headers
    response = requests.get(u"{0}/obp/{1}/my/banks/{2}/accounts/{3}/account".format(BASE_URL, API_VERSION, bank, account), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

# Get owner's transactions
def getTransactions(bank, account):
    response = requests.get(u"{0}/obp/{1}/banks/{2}/accounts/{3}/owner/transactions".format(BASE_URL, API_VERSION, bank, account), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()['transactions']

# Get Transaction by Id.
def getTransaction(bank_id, account_id, transaction_id):
    response = requests.get(u"{0}/obp/{1}/banks/{2}/accounts/{3}/owner/transactions/{4}/transaction".format(BASE_URL, API_VERSION, bank_id, account_id, transaction_id), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

# Get challenge types
def getChallengeTypes(bank, account):
    response = requests.get(u"{0}/obp/{1}/banks/{2}/accounts/{3}/owner/transaction-request-types".format(BASE_URL, API_VERSION, bank, account), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    types = response.json()['transaction_request_types']
    res = []
    for type in types:
      res.append(type['value'])
    return res

# Answer the challenge
def answerChallenge(bank, account, transation_req_id, challenge_query):
    body = '{"id": "' + challenge_query + '","answer": "123456"}'    #any number works in sandbox mode
    response = requests.post(u"{0}/obp/v1.4.0/banks/{1}/accounts/{2}/owner/transaction-request-types/sandbox/transaction-requests/{3}/challenge".format(
         BASE_URL, bank, account, transation_req_id), data=body, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON)
    )
    return response.json()

def getTransactionRequest(bank, account, transation_req_id):
    response = requests.get(u"{0}/obp/{1}/banks/{2}/accounts/{3}/owner/transactions".format(BASE_URL, API_VERSION, bank, account), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json

def initiateTransactionRequest(bank, account, challenge_type, cp_bank, cp_account):
    send_to = {"bank": cp_bank, "account": cp_account}
    payload = '{"to": {"account_id": "' + send_to['account'] +'", "bank_id": "' + send_to['bank'] + \
    '"}, "value": {"currency": "' + OUR_CURRENCY + '", "amount": "' + OUR_VALUE + '"}, "description": "GROUP SETTLEMENT", "challenge_type" : "' + \
    challenge_type + '"}'
    response = requests.post(u"{0}/obp/v1.4.0/banks/{1}/accounts/{2}/owner/transaction-request-types/{3}/transaction-requests".format(BASE_URL, bank, account, challenge_type), data=payload, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

# Create counterparty, input data format: 
# {
#     "name": "test3",
#     "other_account_routing_scheme": "BankAccountID",
#     "other_account_routing_address": "1000203892",
#     "other_bank_routing_scheme": "BankId",
#     "other_bank_routing_address": "00100",
#     "other_branch_routing_scheme": "OBP",
#     "other_branch_routing_address": "Berlin",
#     "is_beneficiary": true
# }

def createCounterparty(bank_id,
                       account_id,
                       name,
                       other_account_routing_scheme,
                       other_account_routing_address,
                       other_bank_routing_scheme,
                       other_bank_routing_address):
    post_data = {
        'name'                         : '%s' % name,
        'other_account_routing_scheme' : '%s' % other_account_routing_scheme ,
        'other_account_routing_address': '%s' % other_account_routing_address,
        'other_bank_routing_scheme'    : '%s' % other_bank_routing_scheme,
        'other_bank_routing_address'   : '%s' % other_bank_routing_address,
        'other_branch_routing_scheme'  : 'OBP', # not useful now, set default value
        'other_branch_routing_address' : 'Berlin', # not useful now, set default value
        'is_beneficiary'               : True
    }
    # Send post request with attached json
    response = requests.post(u"{0}/obp/{1}/banks/{2}/accounts/{3}/owner/counterparties".format(BASE_URL, API_VERSION, bank_id, account_id), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

# Get all entitlements
def getAllEntitlements():
    response = requests.get(u"{0}/obp/{1}/entitlements".format(BASE_URL, API_VERSION), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

# Get user's entitlements
def getEntitlements(user, bank):
    response = requests.get(u"{0}/obp/{1}/banks/{2}/users/{3}/entitlements".format(BASE_URL, API_VERSION, bank, user), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

# Add system role to user
def addRole(role, user):
    post_data = {
        'bank_id'   : '',
        'role_name' : '%s' % role 
    }
    # Send post request with attached json
    response = requests.post(u"{0}/obp/{1}/users/{2}/entitlements".format(BASE_URL, API_VERSION, user), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    # Log result
    return response.text

# Add entitlement to user
def addEntitlement(entitlement, user, bank=''):
    post_data = {
        'bank_id'   : '%s' % bank,
        'role_name' : '%s' % entitlement 
    }
    # Send post request with attached json
    response = requests.post(u"{0}/obp/{1}/users/{2}/entitlements".format(BASE_URL, API_VERSION, user), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    # Log result
    return response.text



# Answer Transaction Request Challenge. - V210
def answerChallengeV210(bank_id,
                        account_id, 
                        transation_req_id, 
                        challenge_type,
                        challenge_query):
    body = '{"id": "' + challenge_query + '","answer": "123456"}'  # any number works in sandbox mode
    response = requests.post( u"{0}/obp/{1}/banks/{2}/accounts/{3}/owner/transaction-request-types/{4}/transaction-requests/{5}/challenge".format(
            BASE_URL, API_VERSION, bank_id, account_id, challenge_type,
            transation_req_id), data=body,
        headers=mergeHeaders(DL_TOKEN, CONTENT_JSON)
    )
    return response.json()

# Create Transaction Request. - V210
# Note : previous called 'initiateTransactionRequest', now keep it the same and OBP-API endpoint name
def createTransactionRequestV210(from_bank_id,
                                 from_account_id,
                                 transaction_request_type,
                                 to_bank_id,
                                 to_account_id,
                                 to_counterparty_id,
                                 to_counterparty_iban):
    if(transaction_request_type== "SANDBOX_TAN"):
        send_to = {"bank": to_bank_id, "account": to_account_id}
        payload = '{"to": {"account_id": "' + send_to['account'] +'", "bank_id": "' + send_to['bank'] + \
                  '"}, "value": {"currency": "' + OUR_CURRENCY + '", "amount": "' + OUR_VALUE + '"}, "description": "Description abc"}'
    elif(transaction_request_type== "SEPA"):
        send_to = {"iban": to_counterparty_iban}
        payload = '{"to": {"iban": "' + send_to['iban'] +\
                  '"}, "value": {"currency": "' + OUR_CURRENCY + '", "amount": "' + OUR_VALUE + '"}, "description": "Description abc", "charge_policy" : "' + \
                  "SHARED" + '"}'
    elif   (transaction_request_type== "COUNTERPARTY"):
        send_to = {"counterparty_id": to_counterparty_id}
        payload = '{"to": {"counterparty_id": "' + send_to['counterparty_id']  + \
                  '"}, "value": {"currency": "' + OUR_CURRENCY + '", "amount": "' + OUR_VALUE + '"}, "description": "Description abc", "charge_policy" : "' + \
                  "SHARED" + '"}'
    else: # FREE_FORM
        send_to = {"bank": to_bank_id, "account": to_account_id}
        payload = '{"to": {"account_id": "' + send_to['account'] +'", "bank_id": "' + send_to['bank'] + \
                  '"}, "value": {"currency": "' + OUR_CURRENCY + '", "amount": "' + OUR_VALUE + '"}, "description": "Description abc", "challenge_type" : "' + \
                  transaction_request_type + '"}'
    response = requests.post(u"{0}/obp/{1}/banks/{2}/accounts/{3}/owner/transaction-request-types/{4}/transaction-requests".format(BASE_URL, API_VERSION, from_bank_id, from_account_id, transaction_request_type), data=payload, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()


# Get Counterparties of one Account.-V220
def getCounterparties(bank, account):
    response = requests.get(u"{0}/obp/{1}/banks/{2}/accounts/{3}/owner/counterparties".format(BASE_URL, API_VERSION, bank, account), headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()['counterparties']


# define some help print transaction methods status
def printMessageNoChallenge(response):
    if "error" in response:
        print("The result is: {0}".format(response))
        # sys.exit("Got an error: " + str(response))
    print("There was no challenge, transaction was created immediately:")
    print("The response is : {0}".format(response))
    print("Transaction status: {0}".format(response['status']))
    print("Transaction id is created: {0}".format(response["transaction_ids"]))


def printMessageWithChallenge(response):
    if "error" in response:
        print("The result is: {0}".format(response))
        # sys.exit("Got an error: " + str(response))
    print("There was a challenge, transaction was interrupted, the transaction_request is 'INITIATED' and new Transaction id is null:")
    print("The response is: {0}".format(response))
    print("Transaction status: {0}".format(response['status']))
    print("New Transaction ID created: {0}".format(response["transaction_ids"]))


def printMessageAfterAnswerChallenge(response):
    if "error" in response:
        print("The result is: {0}".format(response))
        # sys.exit("Got an error: " + str(response))
    print("Transaction is done , and the transaction_request is 'COMPLETED' and new Transaction id is created: :")
    print("The result is: {0}".format(response))
    print("Transaction status: {0}".format(response['status']))
    print("New Transaction ID created: {0}".format(response["transaction_ids"]))


def printGetTransactions(response):
    if "error" in response:
        sys.exit("Got an error: " + str(response))
    print("Print all the transactions : ")       
    count=0
    for transaction in response:
        count=count + 1
        print (str(count) +":"+str(transaction))
        
def printGetTransaction(response, newTransactionId):
    if "error" in response:
        sys.exit("Got an error: " + str(response))
    print("Check wther the new transactionId{0} is exsting".format(newTransactionId))
    print("The result is: {0}".format(response))
    
def printCreateCounterparty(response):
    if "error" in response:
        sys.exit("Got an error: " + str(response))
    print("Counterparty is created:")
    print("The result is: {0}".format(response))

def printGetCounterparties(response):
    if "error" in response:
        sys.exit("Got an error: " + str(response))
    print("Print all the counterparties : ")
    count=0
    for transaction in response:
        count=count + 1
        print (str(count) +":"+str(transaction))

def printGetAccount(response):
    if "error" in response:
        sys.exit("Got an error: " + str(response))
    print("The account detail is: {0}".format(response))        

In [3]:
# Note: in order to use this example, you need to have at least one account
# that you can send money from (i.e. be the owner).
# All properties are now kept in one central place

OUR_BANK     = 'hsbc.01.hk.hsbc'

USERNAME     = 
PASSWORD     = 
CONSUMER_KEY = 

# API server URL
BASE_URL  = "https://openlab.openbankproject.com"
API_VERSION  = "v3.1.0"

# API server will redirect your browser to this URL, should be non-functional
# You will paste the redirect location here when running the script
CALLBACK_URI = 'https://www.hsbc.com.hk'

LOGGING     = True
DL_TOKEN    = { 'Authorization' : 'DirectLogin token=' }
CONTENT_JSON  = { 'content-type'  : 'application/json' }

In [40]:
OUR_CURRENCY = 'EUR'

In [4]:
# Our COUNTERPARTY account id (of the same currency)
OUR_COUNTERPARTY = '8ca8a7e4-6d02-48e3-a029-0b2bf89de9f0'
COUNTERPARTY_BANK = 'hsbc.01.hk.hsbc'
# this following two fields are just used in V210
OUR_COUNTERPARTY_ID = ''
OUR_COUNTERPARTY_IBAN = ''


# Our currency to use
OUR_CURRENCY = 'HKD'

# Our value to transfer
# values below 1000 do not requre challenge request
OUR_VALUE = '0.01'
OUR_VALUE_LARGE = '1000.00'



In [5]:
setBaseUrl(BASE_URL)
setApiVersion(API_VERSION)
login(USERNAME, PASSWORD, CONSUMER_KEY)

Login as {'Authorization': 'DirectLogin username="Robert.Hk.01",password="X!b3726b93",consumer_key="1kcdx3wmhymqntj4ic3d4ea4ns3guroxutb5o1e2"'} to https://openlab.openbankproject.com/my/logins/direct
Received token: eyJhbGciOiJIUzI1NiJ9.eyIiOiIifQ.Ev_vOOxAWPzRWmypS_qvuS-q-yK0fE4qLLkhgtBDJgM


'eyJhbGciOiJIUzI1NiJ9.eyIiOiIifQ.Ev_vOOxAWPzRWmypS_qvuS-q-yK0fE4qLLkhgtBDJgM'

In [4]:
# Get current user
print("")
print(" --- Get current user")
user = getCurrentUser()
print("current user data:\n{0}".format(user))
user_id = user['user_id']
print("current user id: {0}".format(user))

our_bank = OUR_BANK  # banks[0]['id']
print("our bank: {0}".format(our_bank))

# Get accounts for a specific bank
print(" --- Private accounts")

accounts = getPrivateAccounts(our_bank)

for a in accounts:
    print(a['id'])


 --- Get current user
current user data:
{'user_id': 'c623e274-c353-4209-91b9-01523082e9cf', 'email': 'robert.hk.01@x.y', 'provider_id': 'Robert.Hk.01', 'provider': 'https://openlab.openbankproject.com', 'username': 'Robert.Hk.01', 'entitlements': {'list': [{'entitlement_id': '33fdc1b9-6085-447c-9789-63f961a30ecb', 'role_name': 'CanCreateBank', 'bank_id': ''}, {'entitlement_id': '7af1c967-e348-44f0-9e4e-603c1fe7c368', 'role_name': 'CanCheckFundsAvailable', 'bank_id': ''}]}}
current user id: {'user_id': 'c623e274-c353-4209-91b9-01523082e9cf', 'email': 'robert.hk.01@x.y', 'provider_id': 'Robert.Hk.01', 'provider': 'https://openlab.openbankproject.com', 'username': 'Robert.Hk.01', 'entitlements': {'list': [{'entitlement_id': '33fdc1b9-6085-447c-9789-63f961a30ecb', 'role_name': 'CanCreateBank', 'bank_id': ''}, {'entitlement_id': '7af1c967-e348-44f0-9e4e-603c1fe7c368', 'role_name': 'CanCheckFundsAvailable', 'bank_id': ''}]}}
our bank: hsbc.01.hk.hsbc
 --- Private accounts
27dcaa9f-16c3-41c

In [8]:
challenge_types = getChallengeTypes(OUR_BANK, '27dcaa9f-16c3-41c1-b739-a998d9251ca2') 
challenge_type = challenge_types[0]

In [18]:
challenge_types

['SANDBOX_TAN',
 'SEPA',
 'FREE_FORM',
 'COUNTERPARTY',
 'TRANSFER_TO_PHONE',
 'TRANSFER_TO_ACCOUNT',
 'TRANSFER_TO_ATM']

In [9]:
challenge_type

'SANDBOX_TAN'